In [ ]:
import pandas as pd
import numpy as np
#df= pd.read_csv('',dtype=np.float32,converters={'Index':np.uint16},index_col=[0])
id = pd.read_csv('/content/drive/MyDrive/Soft Com Lab/Assignment1/index_id.csv')
id.head(5)
id.loc[id['Student Name']=='Fairuz Nawar']

,Student Id,Student Name,Index Id
30,170104122,Fairuz Nawar,231


# **Dataset Creation**

Loading Dataset

In [ ]:
index_id = 231
df = pd.read_csv('/content/drive/MyDrive/Soft Com Lab/Assignment1/Matrix_Factorization_Assignment.csv')
print("Original Shape :",df.shape)


Original Shape : (45000, 5001)


Dropping Rows

In [ ]:
df.head(5)
df.drop(df[df['Index']%index_id==0].index,inplace=True)
df.shape

(44807, 5001)

Dropping Columns

In [ ]:
ColumnList = df.columns.to_list()
delColumns=[]
for col in ColumnList:
  if col.isdigit() and not int(col)==0 and int(col)%index_id==0:
    delColumns.append(col)

df.drop(columns=delColumns,inplace=True)

print("Created Dataset Shape :",df.shape)

Created Dataset Shape : (44807, 4980)


Filling Null Values

In [ ]:
fdf=df
fdf=fdf.fillna(fdf.mean())
print(fdf.shape)
fdf.head()


(44807, 4980)


,Index,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,4960,4961,4962,4963,4964,4965,4966,4967,4968,4969,4970,4971,4972,4973,4974,4975,4976,4977,4978,4979,4980,4981,4982,4983,4984,4985,4986,4987,4988,4989,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,11071,1.000000,7.0,1.000000,9.000000,6.000000,6.000000,8.000000,4.979157,5.030551,10.000000,4.987798,4.981609,4.969749,4.000000,6.000000,10.000000,0.000000,4.978369,5.000000,4.993285,1.0000,5.000411,4.998775,5.011165,8.000000,4.977525,2.000000,5.028161,5.00026,5.000000,1.000000,9.000000,5.00736,4.978488,4.990944,6.000000,3.00000,5.000507,0.000000,...,1.000000,7.000000,10.0,5.026398,4.996038,4.000000,5.000000,5.008702,7.0,6.0,7.000000,10.000000,8.000000,7.000000,4.993817,5.023786,1.000000,9.000000,5.00711,4.988065,9.00000,5.025133,5.038489,7.000000,1.000000,9.000000,10.000000,1.000000,1.000000,4.994879,2.000000,5.018653,7.000000,9.000000,6.000000,10.00000,7.0,5.026751,5.015137,4.976774
1,19900,5.007433,3.0,3.000000,6.000000,5.011414,3.000000,5.000000,7.000000,9.000000,5.000849,0.000000,8.000000,4.969749,8.000000,4.984697,5.000000,2.000000,4.978369,2.000000,7.000000,4.9916,8.000000,2.000000,3.000000,6.000000,0.000000,5.026503,1.000000,5.00026,4.976034,5.008794,5.000293,5.00736,2.000000,9.000000,6.000000,4.96311,9.000000,4.993239,...,9.000000,5.021526,9.0,5.026398,4.996038,5.010392,6.000000,0.000000,0.0,7.0,0.000000,8.000000,10.000000,0.000000,4.993817,2.000000,0.000000,4.989146,5.00711,8.000000,4.99471,5.025133,5.038489,7.000000,9.000000,6.000000,4.986236,4.978087,7.000000,6.000000,4.937933,5.018653,5.029919,5.000000,9.000000,3.00000,0.0,5.026751,9.000000,2.000000
2,25325,5.000000,7.0,5.006567,9.000000,5.000000,6.000000,5.000000,0.000000,5.030551,10.000000,7.000000,4.981609,4.969749,0.000000,9.000000,4.000000,4.000000,3.000000,5.000945,1.000000,0.0000,5.000411,4.000000,1.000000,4.992243,4.977525,9.000000,7.000000,5.00026,6.000000,5.008794,6.000000,1.00000,4.978488,6.000000,6.000000,4.96311,0.000000,4.993239,...,5.003667,6.000000,2.0,10.000000,4.996038,5.010392,9.000000,5.008702,2.0,2.0,4.982961,9.000000,5.005339,4.996553,1.000000,5.023786,9.000000,2.000000,9.00000,1.000000,4.99471,5.025133,2.000000,8.000000,6.000000,5.029666,4.986236,4.978087,4.987601,4.994879,4.937933,9.000000,10.000000,4.987774,4.998506,9.00000,0.0,5.026751,5.015137,7.000000
3,31006,3.000000,4.0,1.000000,4.997907,5.000000,5.002898,10.000000,4.979157,8.000000,0.000000,4.987798,4.981609,4.000000,1.000000,5.000000,4.964516,3.000000,5.000000,4.000000,3.000000,5.0000,8.000000,7.000000,5.011165,10.000000,6.000000,8.000000,5.000000,5.00026,8.000000,5.008794,4.000000,5.00736,4.978488,7.000000,3.000000,4.96311,5.000507,4.993239,...,10.000000,8.000000,0.0,5.026398,9.000000,5.010392,5.019524,5.008702,2.0,2.0,5.000000,4.991493,5.005339,2.000000,7.000000,5.023786,2.000000,4.989146,2.00000,1.000000,4.00000,4.000000,5.038489,4.981101,4.000000,5.029666,4.986236,4.978087,4.987601,6.000000,3.000000,2.000000,1.000000,3.000000,4.998506,4.98506,2.0,5.026751,4.000000,2.000000
4,37002,2.000000,0.0,5.000000,0.000000,3.000000,5.000000,4.997852,4.979157,5.030551,0.000000,6.000000,4.981609,10.000000,4.979156,4.984697,2.000000,5.010988,8.000000,6.000000,4.993285,4.9916,5.000411,4.998775,10.000000,10.000000,4.977525,5.026503,7.000000,3.00000,6.000000,1.000000,5.000293,1.00000,4.978488,5.000000,5.015219,4.00000,5.000507,6.000000,...,5.003667,0.000000,1.0,1.000000,10.000000,5.010392,10.000000,5.008702,1.0,2.0,10.000000,2.000000,6.000000,9.000000,4.993817,5.023786,5.034202,8.000000,10.00000,4.988065,4.99471,5.000000,8.000000,4.981101,5.055535,6.000000,4.986236,8.000000,10.000000,0.000000,9.000000,5.018653,5.000000,3.000000,5.000000,1.00000,6.0,6.000000,8.000000,4.976774


# **Initialize and Normalize U matrix**

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
import gc
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns

def U_Matrix(n,k):
  scaler = StandardScaler()
  np.random.seed(10)
 
  um= np.single(np.random.rand(n, k))# randomly initialize User Matrix
  um = normalize(um, norm='l2')
  return um

# **Error Calculation**

In [ ]:
def CalculateError(m):
  mseVal= (np.square(np.abs(fdf-m)))
  mseList = mseVal.to_numpy()
  sumVal = np.nansum(mseList)
  dataCount = sum(fdf.count())
  error = sumVal/dataCount
  return(error)
  

# **Implementation**

In [ ]:
def Matrix_factorization(k,n,lamda_u,lamda_v):
  error=0.4
  iteration = 50
  X_axis=[]
  Y_axis=[]
  U_matrix = U_Matrix(n,k)

  for i in range(iteration):
    X_axis.append(i)
    print("\n\n     Iteration",i+1)
    V_matrix= np.single(np.dot(np.linalg.inv(np.dot(U_matrix.T,U_matrix)+lamda_V*np.identity(k)),(np.dot(fdf.T,U_matrix)).T))
    U_matrix= np.single((np.dot(np.linalg.inv(np.dot(V_matrix,V_matrix.T)+lamda_U*np.identity(k)),(np.dot(fdf,V_matrix.T)).T)).T)
    X_matrix= np.dot(U_matrix,V_matrix)
    mse= CalculateError(X_matrix)
    if mse<error:
      print(mse,'-----',error)
      Y_axis.append(error)
      break
    elif mse>=error:
      print(mse,'-----',error)
      Y_axis.append(mse)
  return X_axis,Y_axis,U_matrix,V_matrix,X_matrix
 

# **Initialization and Result Generation**

In [ ]:
import math as m
k=5000
N=fdf.shape[0]
print("N:",N,"K:",k)
lamda_U=(0.00015 + 0.0001*(index_id % 8))
lamda_V=(0.00025 + 0.0001*(index_id % 7))
X_axis,Y_axis,u_matrix,v_matrix,x_matrix = Matrix_factorization(k,N,lamda_U,lamda_V)


### **Converting the output matrix to Dataframe**

In [ ]:
outputXMatrix = pd.DataFrame(x_matrix,index=fdf.index)
outputXMatrix = outputXMatrix.sort_index()
outputXMatrix.shape

In [ ]:
outputXMatrix.shape
outputXMatrix.to_csv('OutputMatrix.csv')
from google.colab import files
files.download('OutputMatrix.csv')

## Plots

In [ ]:
plt.plot(X_axis,Y_axis,color = 'maroon',label="K="+str(k))
plt.legend()
plt.xlabel("Iteration")
plt.ylabel("MSE")
plt.legend()
plt.grid(color = 'gray', linestyle = 'solid', linewidth = 0.5)
plt.tight_layout()
from google.colab import files
plt.savefig("abc.png",dpi=199)
files.download("abc.png") 

# **Movie Recommendation**

In [ ]:
import math as m

def RecommendMovies(x_matrix,user):
    movie=[]
    rating=[]
    realDataset = df.iloc[user-1] 
    IndexList = realDataset.index.to_list()
    for i in range(1, len(IndexList)):
      IndexList[i] = int(IndexList[i])
    orginalMatrix = list(zip(IndexList,realDataset))
    suggestedMatrix = x_matrix.iloc[user-1]
    IndexList_XMatrix = x_matrix.index.to_list()
    predicted = list(zip(IndexList_XMatrix,suggestedMatrix))
    for i in range (len(IndexList)):
      if m.isnan(orginalMatrix[i][1]):
        movie.append(predicted[i-1][0])
        rating.append(predicted[i][1])
        
    print("5 Movie suggestion for User",user,"is :")
    noOfMovies=5
    MovieWithRating=list(zip(movie,rating))
    Suggestion = sorted(MovieWithRating, key = lambda x: x[1], reverse = True)[:noOfMovies]
    for i, mv in enumerate(Suggestion, start=1):
      print(i,". ", mv[0], mv[1])


In [ ]:
UserList = df.index.tolist()
X = pd.DataFrame(x_matrix,index=fdf.index)
X = X.sort_index()

In [ ]:
user=1050
if(user in UserList):
  RecommendMovies(X,user)
else:
  print("User Not Found")

# **Cosine Similarity**

In [ ]:
def CosineSimilarityUsers(U):
  p = cosine_similarity(U.T)
  plt.figure(figsize = (20,20))
  hu=sns.heatmap(p,annot=True)
def CosineSimilarityMovies(V):
  q = cosine_similarity(V)
  plt.figure(figsize = (20,20))
  hm=sns.heatmap(q,annot=True)


In [ ]:
CosineSimilarityUsers(u_matrix)
CosineSimilarityMovies(v_matrix)